In [1]:
# Si no tenemos el módulo instalado
!pip install google-cloud-secret-manager
!pip install langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 2.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 9.3 MB/s e

### LangChain: Modelos, Prompts and Parsers de salida


In [6]:
# Primero autenticamos el usuario :
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

from google.cloud import secretmanager

# Creamos un Cliente de SecretManager:
client = secretmanager.SecretManagerServiceClient()
secret_name = "openai-token"
project_id = '196177837977'

# Contruimos a F-String:
resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"

# Obtenemos el secreto :
response = client.access_secret_version(request={"name": resource_name})
secret_string = response.payload.data.decode('UTF-8')


In [5]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
engine = "gpt-3.5-turbo"
#Cargar variables de entorno
load_dotenv()

# Configurar el motor de OpenAI
engine = "gpt-3.5-turbo"
api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(temperature=0.7, model=engine,openai_api_key=api_key)


## Plantilla de Prompt

In [6]:
estilo_formal = """Español Castellano \
maleducado y grosero.
"""

In [7]:
plantilla = """Transforma el texto \
delimitado por acento grave triple \
en un texto con el estilo {estilo}. \
text: ```{texto}```
"""

In [8]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(plantilla)
# prompt_template.messages[0].prompt
# prompt_template.messages[0].prompt.input_variables

In [9]:
texto_original = """
Bueno, la verdad es que paso bastante de los resultados.
Me importa un bledo lo que piensen en dirección. La próxima ver que me contacten
para hablar de esto, les voy a mandar al carajo!
"""

In [10]:
messages = prompt_template.format_messages(
                    estilo=estilo_formal,
                    texto=texto_original)


In [11]:
print(type(messages))
print(type(messages[0]))
print(messages[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content='Transforma el texto delimitado por acento grave triple en un texto con el estilo Español Castellano maleducado y grosero.\n. text: ```\nBueno, la verdad es que paso bastante de los resultados.\nMe importa un bledo lo que piensen en dirección. La próxima ver que me contacten\npara hablar de esto, les voy a mandar al carajo!\n```\n'


In [12]:
###Llamamos al LLM para transformar el mensaje según la plantilla

response = chat.invoke(messages)
print(response)

content='Bueno, la verdad es que me cago en los resultados.\nMe importa una mierda lo que piensen en dirección. La próxima puta vez que me contacten\npara hablar de esta mierda, les voy a mandar a la mierda, joder!'


### Procesador de salida (output parsers)
Ayudan a transformar la salida cruda del modelo en algo más utilizable o en un formato específico que sea más fácil de manejar en la aplicación donde se utiliza el modelo.

In [13]:
{
  "difícil": False,
  "profesor": "Muy bueno!",
  "calidad_precio": "buena relación calidad-precio"
}

{'difícil': False,
 'profesor': 'Muy bueno!',
 'calidad_precio': 'buena relación calidad-precio'}

In [19]:
valoracion = """\
Me ha gustado mucho el curso. El profesor ha sido muy profesional
y se notaba su experiencia y grandes conocimientos.
A pesar de ello, era la primera vez que estudiaba estos conceptos
y me costó un poco asimilarlos.
Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento.
Lo recomendaré a otros colegas de trabajo!
"""

plantilla = """\
Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil
Contesta True si fue difícil, False si no lo fue o no se indica.

profesor: El profesor cumplió con sus expectativas?
Valoralo el sentimiento sobre el profesor del 1 al 5.
Si no se menciona nada sobre el profesor devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio,\
y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
profesor
calidad_precio

text: {text}
"""

In [20]:
from langchain.prompts import ChatPromptTemplate

prompt_plantilla = ChatPromptTemplate.from_template(plantilla)
print(prompt_plantilla)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Del siguiente texto, extrae la siguiente información:\n\ndificultad: El curso le resulto fácil o difícil\nContesta True si fue difícil, False si no lo fue o no se indica.\n\nprofesor: El profesor cumplió con sus expectativas?\nValoralo el sentimiento sobre el profesor del 1 al 5.\nSi no se menciona nada sobre el profesor devuelve -1.\n\ncalidad_precio: Extrae cualquier referencia a la calidad o precio,y relacionalos ofreciendo el sentimiento.\n\nFormatea la salida como JSON con las siguientes claves:\ndificultad\nprofesor\ncalidad_precio\n\ntext: {text}\n'))]


In [23]:
messages = prompt_plantilla.format_messages(text=valoracion)

chat = ChatOpenAI(temperature=0.7, openai_api_key=secret_string)
response = chat.invoke(messages)
print(response.content)

{
  "dificultad": true,
  "profesor": 5,
  "calidad_precio": "inversión de dinero en mi conocimiento"
}


In [24]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [26]:
dificultad_schema = ResponseSchema(name="dificultad", description="¿El curso le resulto fácil o difícil? Contesta True si fue difícil, False si no lo fue o no se indica.")
profesor_schema = ResponseSchema(name="profesor", description="El profesor cumplió con sus expectativas? Valoralo el sentimiento sobre el profesor del 1 al 5. Si no se menciona nada sobre el profesor devuelve -1.")
calidad_precio_schema = ResponseSchema(name="calidad_precio", description="Extrae cualquier referencia a la calidad o el precio y relaciónalos ofreciendo el sentimiento.")

response_schemas = [dificultad_schema,
                    profesor_schema,
                    calidad_precio_schema]

In [27]:
procesador_salida = StructuredOutputParser.from_response_schemas(response_schemas)

In [28]:
format_instructions = procesador_salida.get_format_instructions()

In [29]:
plantilla2 = """\
Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil. Contesta True si fue difícil, False si no lo fue o no se indica.

profesor: El profesor cumplió con sus expectativas? Valoralo el sentimiento sobre el profesor del 1 al 5. Si no se menciona nada sobre el profesor devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
profesor
calidad_precio

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=plantilla2)

messages = prompt.format_messages(text=valoracion, format_instructions=format_instructions)

In [30]:
print(messages[0].content)

Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil. Contesta True si fue difícil, False si no lo fue o no se indica.

profesor: El profesor cumplió con sus expectativas? Valoralo el sentimiento sobre el profesor del 1 al 5. Si no se menciona nada sobre el profesor devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
profesor
calidad_precio

text: Me ha gustado mucho el curso. El profesor ha sido muy profesional
y se notaba su experiencia y grandes conocimientos.
A pesar de ello, era la primera vez que estudiaba estos conceptos
y me costó un poco asimilarlos.
Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento.
Lo recomendaré a otros colegas de trabajo!


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```js

In [33]:
response = chat.invoke(messages)

In [34]:
print(response.content)

```json
{
	"dificultad": true,
	"profesor": 5,
	"calidad_precio": "positivo"
}
```


In [35]:
output_dict = procesador_salida.parse(response.content)

In [36]:
output_dict

{'dificultad': True, 'profesor': 5, 'calidad_precio': 'positivo'}